# Capacitor sourcing workbook

Use this workflow to build a robust standard capacitor library that is resilient to marketplace fluctuation.

In [38]:
%load_ext autoreload
import sys
import octopart
sys.path.append("../")
import scottopart.constants
import scottopart.mpnmagic.capacitors
import pandas as pd

octopart.logger.setLevel(20)  # disable annoying debug logs

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Step 1. Collect capacitor availability data from Octopart.

In [60]:
# Configure the query here.
AUTHORIZED_CAPACITOR_MANUFACTURERS = ['TDK', 'Murata', 'Yageo', 'KEMET', 'Taiyo Yuden', 'Samsung', 'AVX']
AUTHORIZED_SELLERS = ['Digi-Key', 'Newark', 'Mouser', 'Avnet', 'Arrow Electronics, Inc.']
AUTHORIZED_PACKAGING = ['Cut Tape', 'Tape & Reel', 'None']
query_package = ['0201', '0402', '0603']
query_capacitance = scottopart.constants.CAPACITOR_STANDARD_VALUES

# Convenience functions follow.
def search_octopart_for_capacitors(case_package, capacitance):
    """Search Octopart for capacitors.  Return PartsSearchResult."""
    # We would do this if we could:
    #     filter = {
    #         'manufacturer.displayname' : ['Murata', 'TDK', 'etc.']
    #         'specs.case_package.value' : '0603',
    #         'specs.capacitance.value'  : '4.7e-9'
    #     }
    #     r = octopart.search(query='', limit=100, filter_fields=filter, include_specs=True)
    # But the octopart Python API won't accept a list for manufacturer.displayname.
    # So this is our ugly workaround.
    c = octopart.client.OctopartClient()
    params = {
            'q': '', # leave blank
            'filter[fields][manufacturer.displayname][]': AUTHORIZED_CAPACITOR_MANUFACTURERS,
            'filter[fields][specs.capacitance.value][]': capacitance,
            'filter[fields][specs.case_package.value][]': case_package,
            'limit' : 100,
            'include[]' : 'specs',
    }
    r = octopart.models.PartsSearchResult(c._request('/parts/search', params=params))
    return r

# TODO: pick up authorized_* from a more sensible location
# TODO: figure out Digi-Key double-counting of in-stock quantity
def authorized_in_stock_qty(part):
    """Useful as a sorting key for a list of Part objects."""
    filtered_offers = [o for o in part.offers if o.seller in AUTHORIZED_SELLERS and o.packaging in AUTHORIZED_PACKAGING]
    return sum([o.in_stock_quantity for o in filtered_offers])

In [65]:
# Run the query.
print('Collecting data from Octopart...')
resultlist = []
for case_package in query_package:
    print('--- {} package ---'.format(case_package))
    for capacitance in query_capacitance:
        print(capacitance, end=' ')
        r = search_octopart_for_capacitors(case_package, capacitance)
        sorted_parts = sorted(r.parts, key=authorized_in_stock_qty, reverse=True)
        for p in sorted_parts:
            row = {}
            row['case_package'] = case_package
            row['capacitance'] = capacitance
            row['manufacturer'] = p.manufacturer
            row['mpn'] = p.mpn
            row['authorized_in_stock_qty'] = authorized_in_stock_qty(p)
            magic = scottopart.mpnmagic.capacitors.parse_mpn(p.mpn)
            if magic:
                row['manufacturer_series'] = magic.get('manufacturer_series', None)
                row['dielectric_characteristic'] = magic.get('dielectric_characteristic', None)
                row['voltage_rating_dc'] = magic.get('voltage_rating_dc', None)
                row['capacitance'] = magic.get('capacitance', capacitance)
                row['capacitance_tolerance'] = magic.get('capacitance_tolerance', None)
            resultlist.append(row)
    print('')
print('Done.')
df = pd.DataFrame(resultlist)
print('Results are in DataFrame "df".')

--- 0201 package ---
1e-12 1.5e-12 2.2e-12 3.3e-12 4.7e-12 6.8e-12 1e-11 1.2e-11 1.5e-11 1.8e-11 2.2e-11 2.7e-11 3.3e-11 3.9e-11 4.7e-11 5.6e-11 6.8e-11 8.2e-11 1e-10 1.2e-10 1.5e-10 1.8e-10 2.2e-10 2.7e-10 3.3e-10 3.9e-10 4.7e-10 5.6e-10 6.8e-10 8.2e-10 1e-09 1.2e-09 1.5e-09 1.8e-09 2.2e-09 2.7e-09 3.3e-09 3.9e-09 4.7e-09 5.6e-09 6.8e-09 8.2e-09 1e-08 1.2e-08 1.5e-08 1.8e-08 2.2e-08 2.7e-08 3.3e-08 3.9e-08 4.7e-08 5.6e-08 6.8e-08 8.2e-08 1e-07 1.2e-07 1.5e-07 1.8e-07 2.2e-07 2.7e-07 3.3e-07 3.9e-07 4.7e-07 5.6e-07 6.8e-07 8.2e-07 1e-06 2.2e-06 4.7e-06 1e-05 2.2e-05 4.7e-05 0.0001 
--- 0402 package ---
1e-12 1.5e-12 2.2e-12 3.3e-12 4.7e-12 6.8e-12 1e-11 1.2e-11 1.5e-11 1.8e-11 2.2e-11 2.7e-11 3.3e-11 3.9e-11 4.7e-11 5.6e-11 6.8e-11 8.2e-11 1e-10 1.2e-10 1.5e-10 1.8e-10 2.2e-10 2.7e-10 3.3e-10 3.9e-10 4.7e-10 5.6e-10 6.8e-10 8.2e-10 1e-09 1.2e-09 1.5e-09 1.8e-09 2.2e-09 2.7e-09 3.3e-09 3.9e-09 4.7e-09 5.6e-09 6.8e-09 8.2e-09 1e-08 1.2e-08 1.5e-08 1.8e-08 2.2e-08 2.7e-08 3.3e-08 3.9e-08 

In [67]:
# Save dataframe to CSV.
df.to_csv('capacitors-2018-09-05.csv')

## Step 2. Analyze!

In [ ]:
# (optional)
# Read dataframe from CSV (in case you don't want to re-run the query.)
df = pd.read_csv('capacitors-2018-09-05.csv', index_col=[0], dtype={'case_package' : object})

In [84]:
df.groupby(['case_package', 'capacitance']).head(5)

,authorized_in_stock_qty,capacitance,capacitance_tolerance,case_package,dielectric_characteristic,manufacturer,manufacturer_series,mpn,voltage_rating_dc
0,345764,1.000000e-12,±0.1pF,0201,C0G/NP0,Murata,GJM,GJM0335C1E1R0BB01D,25
1,300495,1.000000e-12,±0.1pF,0201,C0G/NP0,Samsung,CL,CL03C010BA3GNNC,25
2,229355,1.000000e-12,±0.05pF,0201,C0G/NP0,Murata,GJM,GJM0335C1E1R0WB01D,25
3,170586,1.000000e-12,±0.25pF,0201,C0G/NP0,TDK,C,C0603C0G1E010C030BA,25
4,95982,1.000000e-12,±0.1pF,0201,C0G/NP0,AVX,None,02013A1R0BAT2A,25
98,152020,1.500000e-12,±0.1pF,0201,C0G/NP0,Murata,GRM,GRM0335C1H1R5BA01D,50
99,147628,1.500000e-12,±0.25pF,0201,C0G/NP0,Murata,GJM,GJM0335C1E1R5CB01D,25
100,69481,1.500000e-12,±0.05pF,0201,C0G/NP0,Murata,GJM,GJM0335C1E1R5WB01D,25
101,48549,1.500000e-12,±0.25pF,0201,C0G/NP0,TDK,C,C0603C0G1E1R5C030BA,25
102,43305,1.500000e-12,±0.25pF,0201,CH,TDK,C,C0603CH1E1R5C030BA,25
